In [1]:
# switch to the project directory
%cd ..
# working directory should be ../FSE

c:\Users\admin\Desktop\research\pdi


In [2]:
import sys
import os
module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pdi.constants import (
    PARTICLES_DICT,
)

from pdi.data.types import Split

In [4]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
device = torch.device("cpu")

In [5]:
from pdi.data.preparation import EnsemblePreparation
from pdi.models import NeuralNetEnsemble

wandb_config = {
    "bs": 64,
    "max_epochs": 1,  # 40
    "dropout": 0.1,
    "gamma": 0.9,
    "patience": 5,
    "patience_threshold": 0.001,
    "h0": 64,
    "h1": 32,
    "h2": 16,
    "start_lr": 5e-4,
}

data_preparation = EnsemblePreparation()
create_model = lambda: NeuralNetEnsemble(
    data_preparation.get_group_ids(),
    [wandb.config.h0, wandb.config.h1, wandb.config.h2, 1],
    nn.ReLU,
    wandb.config.dropout,
).to(device)


In [6]:
# from pdi.data.preparation import FeatureSetPreparation
# from pdi.models import AttentionModel
# from pdi.data.constants import N_COLUMNS

# wandb_config = {
#     "bs": 64,
#     "max_epochs": 1,  # 40
#     "dropout": 0.1,
#     "gamma": 0.9,
#     "patience": 5,
#     "patience_threshold": 0.001,
#     "embed_in": N_COLUMNS + 1,
#     "embed_hidden": 128,
#     "d_model": 32,
#     "ff_hidden": 128,
#     "pool_hidden": 64,
#     "num_heads": 2,
#     "num_blocks": 2,
#     "start_lr": 2e-4,
# }

# data_preparation = FeatureSetPreparation()
# create_model = lambda: AttentionModel(
#     wandb.config.embed_in,
#     wandb.config.embed_hidden,
#     wandb.config.d_model,
#     wandb.config.ff_hidden,
#     wandb.config.pool_hidden,
#     wandb.config.num_heads,
#     wandb.config.num_blocks,
#     nn.ReLU,
#     wandb.config.dropout,
# ).to(device)


In [7]:
import wandb
import os
from torch.profiler import profile, ProfilerActivity
from pdi.data.constants import GROUP_ID_KEY

experiment_name = "Profile"
(train_loader,) = data_preparation.prepare_dataloaders(wandb_config["bs"], 0, [Split.TRAIN])
target_code = 2212

save_path = f"reports/profile"
os.makedirs(save_path, exist_ok=True)


def trace_handler(p):
    print(p.key_averages(group_by_stack_n=5).table(row_limit=-1, max_name_column_width=120, sort_by="self_cpu_time_total"))
    p.export_stacks(f"{save_path}/cpu_stacks.txt", "self_cpu_time_total")


with wandb.init(project=experiment_name, config=wandb_config, name=PARTICLES_DICT[target_code], mode="offline") as run:
    pos_weight = torch.tensor(data_preparation.pos_weight(target_code)).float().to(device)
    wandb.log({"pos_weight": pos_weight.item()})
    model = create_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.start_lr)
    loss_fun = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    model.train()

    with profile(activities=[ProfilerActivity.CPU], with_stack=True, schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=20),
        on_trace_ready=trace_handler) as p: #with_flops=True ProfilerActivity.CUDA, 
        for i, (input_data, targets, data_dict) in enumerate(train_loader):
            input_data = input_data.to(device)
            binary_targets = (targets == target_code).type(torch.float).to(device)
            optimizer.zero_grad()
            group_id = data_dict.get(GROUP_ID_KEY)
            out = model(input_data, group_id)
            loss = loss_fun(out, binary_targets)
            loss.backward()
            optimizer.step()
            p.step()
            if i >= 22:
                break

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


-------------------------------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                                                                           Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  Source Location                                                              
-------------------------------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                    enumerate(DataLoader)#_SingleProcessDataLoaderIter.__next__        28.03%     100.509ms        50.66%     181.607ms       9.080ms            20  ...method _record_function_enter of PyCapsule object at 0x000002095EBDFC30>  
                            

pos_weight,▁
pos_weight,48.36434
